In [17]:
using LowLevelFEM

In [18]:
gmsh.initialize()
gmsh.open("pml-circle.geo")

mat = Material("surf")
prob = Problem([mat], type=:General, dim=2, fdim=2)

Info    : Reading 'pml-circle.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 30%] Meshing curve 2 (Circle)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 70%] Meshing curve 4 (Line)
Info    : [ 90%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.0010699s, CPU 0.001s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.147498s, CPU 0.145144s)
Info    : 913 nodes 1895 elements
Info    : Done reading 'pml-circle.geo'


Problem("pml-circle", :General, 2, 2, Material[Material("surf", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 913, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [19]:
μ = mat.μ
λ = mat.λ
ρ = mat.ρ

7.85e-9

In [20]:
fs(x, y, z) = √(x^2 + y^2 + z^2) < 10 ? 1 : 1 + (√(x^2 + y^2 + z^2) - 10) * 10

s = scalarField(prob, "surf", fs)

ScalarField
[1.0; 51.0; … ; 35.94477763729036; 25.359466505019665;;]

In [21]:
showDoFResults(s, name="coeff")

0

In [22]:
K = stiffnessMatrixPoisson(prob, coefficient=2μ * s) + gradDivMatrix(prob, coefficient=λ) - curlCurlMatrix(prob, coefficient=μ * s)

sparse([1, 2, 11, 12, 133, 134, 343, 344, 929, 930  …  1185, 1186, 1189, 1190, 1539, 1540, 1787, 1788, 1825, 1826], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1826, 1826, 1826, 1826, 1826, 1826, 1826, 1826, 1826, 1826], [1.4568726434788825e6, 12585.69860693167, -54697.08085822448, 52834.96451581128, -26176.29554182908, -80673.44158960378, -131512.7122053824, -94503.35581761322, -162822.78587267682, 91484.45968767407  …  -828365.2967460868, -1.6218065901507777e6, 87886.1966043503, -3.942726175379038e6, 1.1308748520800793e6, -744583.9953939882, -1.1712601993913199e6, -470002.88856094924, 645275.3392848289, 1.169183479975741e7], 1826, 1826)

In [23]:
elsupp = BoundaryCondition("perimeter", kx=1, ky=1)

BoundaryCondition("perimeter", nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, 1, 1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [24]:
applyElasticSupport!(K, [elsupp])

1826×1826 SparseArrays.SparseMatrixCSC{Float64, Int64} with 24780 stored entries:
⎡⠻⣦⡀⠁⠀⠀⠀⣽⠁⠀⠸⣖⠀⠀⠀⠁⠀⠀⠈⣲⣗⡆⠀⣡⠞⢁⢀⠠⠀⠴⠐⢠⢰⠁⠀⠀⠀⠱⠚⠀⎤
⎢⠄⠈⠻⣦⡀⠀⠀⠻⠀⠀⠨⠃⠀⠀⠀⠀⠀⠀⠀⠚⠗⠇⢀⡌⠳⠀⠈⠁⠉⠀⠒⠚⠸⠠⡠⣠⠀⢐⡀⠀⎥
⎢⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣏⠀⠀⠀⠀⠀⠀⠀⠀⠈⣀⢾⠉⢼⠠⡃⠔⠤⎥
⎢⣄⣤⣤⡀⠀⠈⠻⣦⡄⠀⠀⠀⠀⠀⢀⡤⢤⣠⡤⢠⣠⣤⠘⠆⠀⠀⠀⠀⠀⠀⠀⠐⠁⠀⠛⠿⠄⠆⠑⠀⎥
⎢⠁⠀⠀⠀⠀⠀⠀⠉⣿⣿⣄⠰⣓⢀⠀⢿⡿⣼⠍⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢲⢦⠦⠂⠀⠀⠀⠀⢀⡙⣿⢟⣛⣯⣠⣚⣷⣷⣢⣴⠶⠶⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠙⢘⡿⣼⢵⣷⣿⣿⣿⣶⣟⡳⠠⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠄⠀⠀⠀⠀⠀⠀⡴⣤⣄⣠⢺⣿⣿⡿⣯⣽⣾⣿⣿⠞⠺⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⣳⣛⣯⢽⣿⢻⣿⣳⣿⡿⣯⣿⡷⢗⣿⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢢⣠⣠⠀⠀⠀⠀⣋⠧⠅⢈⣾⢿⡹⣿⣿⢿⡿⡻⣮⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠹⠽⠽⠅⠀⠀⠀⣾⠀⠀⢸⡇⠀⠦⣺⡁⣽⣵⣿⣿⣿⢟⡁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠄⣠⡀⠴⠋⠙⠲⠄⠀⠀⠀⠁⠀⠀⠀⠀⠀⠉⠀⠀⠁⠈⠛⣤⠠⠤⣩⣉⣐⣐⠠⢾⣿⡿⠷⡿⡟⢷⡯⡍⎥
⎢⠞⢁⠙⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡆⢻⣶⡀⠀⠈⠉⠙⢻⣿⢶⢀⠧⠁⣞⣶⢵⎥
⎢⠀⡐⠆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⢺⠀⠈⠻⣦⡀⠀⠀⠸⣬⡿⠀⣧⣯⣧⢳⣇⎥
⎢⢀⡄⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢐⢸⡆⠀⠀⠈⠻⣦⡀⢨⣟⡄⠯⣗⣽⣺⣿⡅⎥
⎢⠐⣀⣸⠀⡀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣆⣷⣀⣀⡀⡀⣈⡿⣯⣷⡇⢱⣵⡿⣾⣝⡱⎥
⎢⠔⠒⠒⡂⣠⣜⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⡿⢻⣟⣦⡿⠛⠽⠽⠿⣻⣾⣈⣿⣽⣟⣿⣾⎥
⎢⠀⠀⠀⣪⣃⣄⣿⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣽⡧⠤⡔⠤⣤⢯⢧⢕⣶⣦⣼⠿⣧⠻⣯⣼⣵⎥
⎢⢄⡀⢀⢀⠤⠢⠠⠅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢿⣍⣡⢤⠯⣿⣳⣻⣻⣯⣷⢿⡿⣦⣟⣽⣘⠛⎥
⎣⠚⠀⠀⠈⠐⡅⠑⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡏⠯⢜⣟⠽⢶⠟⠿⢗⡹⣻⣿⢖⣿⣶⠘⢵⢗⎦

In [25]:
load = BoundaryCondition("source", fx=1)
f = loadVector(prob, [load])
u = solveDisplacement(K, f)

showDoFResults(u, name="disp-test")

1

In [26]:
M = massMatrixPoisson(prob, coefficient=ρ)

sparse([1, 11, 133, 343, 929, 1069, 1523, 2, 12, 134  …  1539, 1787, 1825, 1176, 1182, 1186, 1190, 1540, 1788, 1826], [1, 1, 1, 1, 1, 1, 1, 2, 2, 2  …  1825, 1825, 1825, 1826, 1826, 1826, 1826, 1826, 1826, 1826], [3.442438500868962e-9, 6.000525605489545e-10, 5.67461634931048e-10, 5.476800129505801e-10, 5.572588514768444e-10, 5.734866769349473e-10, 5.964987640265899e-10, 3.442438500868962e-9, 6.000525605489545e-10, 5.67461634931048e-10  …  4.3205942824727097e-10, 3.6648758439086413e-10, 2.5035980740582033e-9, 4.611774551897357e-10, 3.591515978891548e-10, 4.605880108926762e-10, 4.2413399744850225e-10, 4.3205942824727097e-10, 3.6648758439086413e-10, 2.5035980740582033e-9], 1826, 1826)

In [27]:
n = 500

u0 = vectorField(prob, "surf", [0, 0, 0])
v0 = vectorField(prob, "surf", [0, 0, 0])
Tmin = smallestPeriodTime(K, M, BoundaryCondition[])
dt = Tmin / π

ff(x, y, z, t) = (tanh(1e8 * (t - 2dt)) - tanh(1e8 * (t - 3dt)))
#ff(x, y, z, t) = t
fx = ScalarField(prob, "source", ff, steps=n, tmax=n * dt)
O = ScalarField(prob, "source", 0, steps=n, tmax=n * dt)
f = vectorField([fx, O, O])
f = elementsToNodes(f)
f = projectTo2D(f)

VectorField
[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

In [28]:
showDoFResults(f, name="f")

2

In [29]:
u, v = CDM(K, M, f, u0, v0, n, dt)

(VectorField(Matrix{Float64}[], [0.0 1.6653706017439673e-18 … 1.56281587118919e-7 1.596725933544734e-7; 0.0 0.0 … 3.466901702777931e-10 -2.181277178184596e-9; … ; 0.0 -8.874922311398035e-20 … 1.515507974118859e-7 1.506654959446423e-7; 0.0 0.0 … 1.8000766187962223e-9 1.7232017940946033e-9], [0.0, 1.2181787559945157e-8, 2.4363575119890313e-8, 3.654536267983547e-8, 4.8727150239780626e-8, 6.090893779972578e-8, 7.309072535967093e-8, 8.527251291961608e-8, 9.745430047956123e-8, 1.0963608803950638e-7  …  5.969075904373163e-6, 5.981257691933108e-6, 5.9934394794930535e-6, 6.005621267052999e-6, 6.017803054612944e-6, 6.029984842172889e-6, 6.042166629732835e-6, 6.05434841729278e-6, 6.066530204852725e-6, 6.0787119924126704e-6], Int64[], 500, :v2D, Problem("pml-circle", :General, 2, 2, Material[Material("surf", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 913, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothin

In [30]:
showDoFResults(u, name="u")

3

In [31]:
openPostProcessor()

XRequest.18: BadValue 0x0


-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [32]:
gmsh.finalize()